# Excel._add_calculated_values(self)

In [1]:
import os, sys
sys.path.append('/home/dolley/gear/lib')
import gear.datasetuploader as datasetuploader

test_file = '/home/dolley/gear/tests/base_template.xlsx'
dataset = datasetuploader.DatasetUploader.upload_dataset('excel')
# adata = dataset._read_file(test_file)
# adata
dataset._read_file(test_file)
dataset.adata.var.head()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


,gene_symbol
genes,
ENSMUSG00000000567,Gnai3
ENSMUSG00000000568,Pbsn
ENSMUSG00000000569,Cdc45l
ENSMUSG00000000570,H19
ENSMUSG00000000571,Scml2


In [2]:
# adata.var.head()
# adata._add_calculated_values()
adata = dataset.adata
dataset._add_calculated_values(adata)

In [3]:
X = adata.X
X

array([[72., 79., 40., ..., 87., 11., 84.],
       [92., 15., 15., ..., 81., 50.,  4.],
       [51., 66., 94., ..., 41., 75., 58.],
       ...,
       [25.,  7., 60., ..., 79., 83., 74.],
       [40., 82., 49., ..., 82., 70., 80.],
       [81., 14., 43., ...,  6., 34., 18.]], dtype=float32)

In [4]:
obs = adata.obs
obs.head()

,cell_type,condition,replicate,time_point,time_unit
observations,,,,,
obs-ctrl-1-0h,utricle,control,1,0,hour
obs-ctrl-2-0h,utricle,control,2,0,hour
obs-ctrl-3-0h,utricle,control,3,0,hour
obs-trt-1-0h,utricle,treated,1,0,hour
obs-trt-2-0h,utricle,treated,2,0,hour


In [5]:
# 'replicate' column index = 2. Use this if not hardcoding the index
# obs.columns.get_loc('replicate')

In [6]:
#Get replicate column into np.array
# replicates = obs.iloc[:, 2]

#DO THIS INSTEAD of the above 2 lines:
replicates = obs.loc[:, 'replicate']

In [7]:
# Build list of total number of replicates for each group
replicate_count = list()
prior_rep = None
for r in replicates:
    if prior_rep is None or r > prior_rep:
        prior_rep = r
    else:
        replicate_count.append(prior_rep)
        prior_rep = None

replicate_count.append(prior_rep)
    
print(replicate_count)

[3, 3, 3, 3, 3, 3]


In [8]:
replicate_vals = X[:,0] #TODO: iterate over all rows
print(replicate_vals)
print(X.shape)
col_count, row_count = X.shape

[ 72.  92.  51.  93.   1.  46.   0.  33.  46.  75.  56.  28.  90. 100.
   7.  25.  40.  81.]
(18, 100)


In [9]:
#
# THIS ALL WORKS. THE 'In' BELOW IS TRYING TO IMPROVE THE double counter of the inner-most loop (3-20-18)
#
# import numpy as np

# average_list = list()
# std_list = list()                         #TODO TODO Could add this here!
# for x in range(row_count):
#     replicate_vals = X[:,x]
#     averages = list()
#     std_devs = list()                     #TODO TODO Could add this here!
#     start = 0
#     end = 0
#     for total_rep in replicate_count:
#     #     end += total_rep
#     #     print(replicate_vals[start:end])
#     #     start += total_rep
#     #     print("total_rep: ", total_rep, "start: ", total_rep, "end: ", end)
#         end += total_rep

#         values = replicate_vals[start:end]
#         rep_avg = round( (np.nanmean(values)), 4)
#         rep_std = np.nanstd(values)       #TODO TODO Could add this here!
#         for x in range(total_rep):
#             averages.append(rep_avg)
#             std_devs.append(rep_std)
#             x += 1

#         start += total_rep

#     average_list.append(averages)
#     std_list.append(std_devs)           #TODO TODO Could add this here!
#     x += 1

# #Transpose back to match the original .X
# Xmean = np.array(average_list).T
# print(Xmean)
# print(Xmean[:,0])
# print(type(Xmean))
# print(Xmean.shape)

# Xstd = np.array(std_list).T              #TODO TODO Could add this here!
# print(Xstd)                              #TODO TODO Could add this here!

In [10]:
import numpy as np
import scipy.stats as stats


average_list = list()
std_list = list()
sem_list = list()

replicate_col = 'replicate'

obs = adata.obs
X = adata.X
col_count, row_count = X.shape

# Get replicate column
replicates = obs.loc[:, replicate_col]

# Build list of total number of replicates for each group
replicate_count = list()
prior_rep = None
for r in replicates:
    if prior_rep is None or r > prior_rep:
        prior_rep = r
    else:
        replicate_count.append(prior_rep)
        prior_rep = None

#Gets last replicate count
replicate_count.append(prior_rep)
print('replicate_count list: ', replicate_count)

#Override replicate list to test variable replicate numbers 
# replicate_count = [3, 3, 3, 3, 3, 3]
## replicate_count = [3, 2, 1, 4, 4, 4]
## print('New replicate_count list: ', replicate_count)

for row in range(row_count):
    replicate_vals = X[:,row]
    
    averages = list()
    stds = list()
    sems = list()
    
    start = 0
    end = 0
    for total_rep in replicate_count:
        end += total_rep

        # Subsample the replicates from 1 condition
        values = replicate_vals[start:end]
        rep_avg = round( np.nanmean(values), 4 )
        rep_std = round( np.nanstd(values), 4 )
        rep_sem = round( stats.sem(values, nan_policy="omit"), 4 )

        # Add the average, std, and sem for each replicate
        for r in range(total_rep):
            averages.append(rep_avg)
            stds.append(rep_std)
            sems.append(rep_sem)
            r += 1

#         start += total_rep
#         print('start: ', start, ' end: ', end, ' values: ', values)
        start = end

    average_list.append(averages)
    std_list.append(stds)
    sem_list.append(sems)
    print(averages)
    print(stds)
    print(sems)
    row += 1

replicate_count list:  [3, 3, 3, 3, 3, 3]
[71.6667, 71.6667, 71.6667, 46.6667, 46.6667, 46.6667, 26.3333, 26.3333, 26.3333, 53.0, 53.0, 53.0, 65.6667, 65.6667, 65.6667, 48.6667, 48.6667, 48.6667]
[16.7398, 16.7398, 16.7398, 37.5618, 37.5618, 37.5618, 19.362, 19.362, 19.362, 19.3046, 19.3046, 19.3046, 41.684, 41.684, 41.684, 23.669, 23.669, 23.669]
[11.8369, 11.8369, 11.8369, 26.5602, 26.5602, 26.5602, 13.691, 13.691, 13.691, 13.6504, 13.6504, 13.6504, 29.475, 29.475, 29.475, 16.7365, 16.7365, 16.7365]
[53.3333, 53.3333, 53.3333, 77.0, 77.0, 77.0, 42.0, 42.0, 42.0, 42.3333, 42.3333, 42.3333, 61.3333, 61.3333, 61.3333, 34.3333, 34.3333, 34.3333]
[27.6204, 27.6204, 27.6204, 2.9439, 2.9439, 2.9439, 22.8619, 22.8619, 22.8619, 24.2808, 24.2808, 24.2808, 12.6579, 12.6579, 12.6579, 33.8264, 33.8264, 33.8264]
[19.5306, 19.5306, 19.5306, 2.0817, 2.0817, 2.0817, 16.1658, 16.1658, 16.1658, 17.1691, 17.1691, 17.1691, 8.9505, 8.9505, 8.9505, 23.9188, 23.9188, 23.9188]
[49.6667, 49.6667, 49.6667, 54.

[69.6667, 69.6667, 69.6667, 36.6667, 36.6667, 36.6667, 35.6667, 35.6667, 35.6667, 40.6667, 40.6667, 40.6667, 41.6667, 41.6667, 41.6667, 55.6667, 55.6667, 55.6667]
[20.4179, 20.4179, 20.4179, 6.1283, 6.1283, 6.1283, 16.4181, 16.4181, 16.4181, 41.2661, 41.2661, 41.2661, 9.3927, 9.3927, 9.3927, 35.141, 35.141, 35.141]
[14.4376, 14.4376, 14.4376, 4.3333, 4.3333, 4.3333, 11.6094, 11.6094, 11.6094, 29.1795, 29.1795, 29.1795, 6.6416, 6.6416, 6.6416, 24.8484, 24.8484, 24.8484]
[45.3333, 45.3333, 45.3333, 45.3333, 45.3333, 45.3333, 57.6667, 57.6667, 57.6667, 60.0, 60.0, 60.0, 43.3333, 43.3333, 43.3333, 62.3333, 62.3333, 62.3333]
[26.3354, 26.3354, 26.3354, 26.4491, 26.4491, 26.4491, 32.7143, 32.7143, 32.7143, 22.5536, 22.5536, 22.5536, 38.3087, 38.3087, 38.3087, 20.7257, 20.7257, 20.7257]
[18.622, 18.622, 18.622, 18.7023, 18.7023, 18.7023, 23.1325, 23.1325, 23.1325, 15.9478, 15.9478, 15.9478, 27.0883, 27.0883, 27.0883, 14.6553, 14.6553, 14.6553]
[48.6667, 48.6667, 48.6667, 55.0, 55.0, 55.0, 71.

In [11]:
################## TEST datasetstats.py DatasetStats.get_replicate_averages() ####################################

In [12]:
import os, sys
sys.path.append('/home/dolley/gear/lib')
import gear.datasetuploader as datasetuploader

#Read in xlsx file and convert it to AnnData object
#Automatically appends AnnData object to DatasetUploader object as .adata
#  access adata via .adata.var, .adata.obs, adata.X
test_file = '/home/dolley/gear/tests/base_template.xlsx'
dataset = datasetuploader.DatasetUploader.upload_dataset('excel')
dataset._read_file(test_file)
dataset.adata.var.head()

,gene_symbol
genes,
ENSMUSG00000000567,Gnai3
ENSMUSG00000000568,Pbsn
ENSMUSG00000000569,Cdc45l
ENSMUSG00000000570,H19
ENSMUSG00000000571,Scml2


In [13]:
from gear.datasetstats import DatasetStats

Xmean = DatasetStats.get_replicate_averages(dataset.adata)
Xmean

array([[71.6667, 53.3333, 49.6667, ..., 69.6667, 45.3333, 48.6667],
       [71.6667, 53.3333, 49.6667, ..., 69.6667, 45.3333, 48.6667],
       [71.6667, 53.3333, 49.6667, ..., 69.6667, 45.3333, 48.6667],
       ...,
       [48.6667, 34.3333, 50.6667, ..., 55.6667, 62.3333, 57.3333],
       [48.6667, 34.3333, 50.6667, ..., 55.6667, 62.3333, 57.3333],
       [48.6667, 34.3333, 50.6667, ..., 55.6667, 62.3333, 57.3333]],
      dtype=float32)

In [14]:
# dataset.adata.Xmean = Xmean
# dataset.adata.Xmean

In [15]:
dataset._add_calculated_values(dataset.adata)
dataset.adata.Xmean

array([[71.6667, 53.3333, 49.6667, ..., 69.6667, 45.3333, 48.6667],
       [71.6667, 53.3333, 49.6667, ..., 69.6667, 45.3333, 48.6667],
       [71.6667, 53.3333, 49.6667, ..., 69.6667, 45.3333, 48.6667],
       ...,
       [48.6667, 34.3333, 50.6667, ..., 55.6667, 62.3333, 57.3333],
       [48.6667, 34.3333, 50.6667, ..., 55.6667, 62.3333, 57.3333],
       [48.6667, 34.3333, 50.6667, ..., 55.6667, 62.3333, 57.3333]],
      dtype=float32)

In [16]:
############################  TEST DatasetStats.get_replicate_std() ###############################################
Xstd = DatasetStats.get_replicate_std(dataset.adata)
Xstd
print(Xstd[:,0])

[16.7398 16.7398 16.7398 37.5618 37.5618 37.5618 19.362  19.362  19.362
 19.3046 19.3046 19.3046 41.684  41.684  41.684  23.669  23.669  23.669 ]


In [17]:
############################  TEST DatasetStats.get_replicate_sem() ###############################################
Xsem = DatasetStats.get_replicate_sem(dataset.adata)
Xsem
print(Xsem[:,0])

[11.8369 11.8369 11.8369 26.5602 26.5602 26.5602 13.691  13.691  13.691
 13.6504 13.6504 13.6504 29.475  29.475  29.475  16.7365 16.7365 16.7365]


In [18]:
def get_replicate_counts(obs=None):

    # Get replicate counts by grouping them with pandas built-ins
    if 'time_point' in obs.columns:
        groups = obs.groupby(['cell_type', 'condition', 'time_point']).count().reset_index()
    else:
        groups = obs.groupby(['cell_type', 'condition']).count().reset_index()

    #pandas series containing number of replicates for each condition
    replicate_count = groups.loc[:, 'replicate']

    return replicate_count

In [19]:
###################################### Calculate p-values #########################################################
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html#scipy.stats.pearsonr
# pearson_coefficient, pvalue = stats.pearsonr(control_array, treated_array)
import  scipy.stats as stats
X = adata.X
obs = adata.obs
obs

,cell_type,condition,replicate,time_point,time_unit
observations,,,,,
obs-ctrl-1-0h,utricle,control,1,0,hour
obs-ctrl-2-0h,utricle,control,2,0,hour
obs-ctrl-3-0h,utricle,control,3,0,hour
obs-trt-1-0h,utricle,treated,1,0,hour
obs-trt-2-0h,utricle,treated,2,0,hour
obs-trt-3-0h,utricle,treated,3,0,hour
obs-ctrl-1-24h,utricle,control,1,24,hour
obs-ctrl-2-24h,utricle,control,2,24,hour
obs-ctrl-3-24h,utricle,control,3,24,hour


In [20]:
replicate_counts = get_replicate_counts(obs)
replicate_counts

0    3
1    3
2    3
3    3
4    3
5    3
Name: replicate, dtype: int64

In [21]:
rows, cols = obs.shape
print(rows, 'rows x ', cols, 'cols')

18 rows x  5 cols


In [22]:
### USE THIS TO GENERATE REPLICATE COUNTS IN DatasetStats.get_replicate_counts()
# groups = obs.groupby(['cell_type', 'condition', 'time_point']).count().reset_index()
# if 'time_point' in obs.columns:
#     print('time_point column found')
#     groups = obs.groupby(['cell_type', 'condition', 'time_point']).count().reset_index()
# else:
#     print('No time_point column found')
#     groups = obs.groupby(['cell_type', 'condition']).count().reset_index()
# print(groups)
# replicates = groups.loc[:, 'replicate']
## for i, r in replicates.iteritems():
##     print(r)

In [23]:
groupby_attrs = ['cell_type', 'condition', 'time_point']
# grouped = obs.groupby(['cell_type', 'condition', 'time_point']).count()
grouped = obs.groupby([groupby_attrs[0], groupby_attrs[1], groupby_attrs[2]]).count()
print(grouped.head())
# grouped.columns

                                replicate  time_unit
cell_type condition time_point                      
utricle   control   0                   3          3
                    24                  3          3
                    48                  3          3
          treated   0                   3          3
                    24                  3          3


In [24]:
grouped.reset_index().T
grouped
# grouped.loc['control']
# grouped[['cell_type','']]
# grouped.index.get_level_values(0)
# grouped.iloc[0]

replicate  time_unit
cell_type condition time_point                      
utricle   control   0                   3          3
                    24                  3          3
                    48                  3          3
          treated   0                   3          3
                    24                  3          3
                    48                  3          3

In [25]:
# http://pandas.pydata.org/pandas-docs/stable/advanced.html#advanced-indexing-with-hierarchical-index
grouped.loc['utricle','control', 0]['replicate']

3

In [26]:
level_1_len = len(grouped.index[0])
# print(level_1_len, grouped.index[0])
print(grouped.index.levels[1].size)
for row in grouped.itertuples():
    print(grouped.index[0][0], row[1])
print(grouped.index)

2
utricle 3
utricle 3
utricle 3
utricle 3
utricle 3
utricle 3
MultiIndex(levels=[['utricle'], ['control', 'treated'], [0, 24, 48]],
           labels=[[0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]],
           names=['cell_type', 'condition', 'time_point'])


In [27]:
print(X)
X.shape

[[72. 79. 40. ... 87. 11. 84.]
 [92. 15. 15. ... 81. 50.  4.]
 [51. 66. 94. ... 41. 75. 58.]
 ...
 [25.  7. 60. ... 79. 83. 74.]
 [40. 82. 49. ... 82. 70. 80.]
 [81. 14. 43. ...  6. 34. 18.]]


(18, 100)

In [28]:
obs = adata.obs
X_cols, X_rows = X.shape
obs_rows, obs_cols = obs.shape

total_conditions = grouped.index.levels[0].size * grouped.index.levels[1].size * grouped.index.levels[2].size

# print(grouped.index.levels[0].name, ': ', grouped.index.levels[0][0], ': ', grouped.index.levels[0].size)
# print(grouped.index.levels[1].name, ': ', grouped.index.levels[1].values, ': ', grouped.index.levels[1].size)
# print(grouped.index.levels[2].name, ': ', grouped.index.levels[2].values, ': ', grouped.index.levels[2].size)
# print("number of conditions: {0}".format(total_conditions))

replicate_list = get_replicate_counts(adata.obs)
# print(replicate_list)

# for r in range(X_rows):

control_string = ['control', 'wildtype', 'input']

p_values_list = list()
for g, gene in enumerate(X.T):
#     print(gene)
    
#     p_values = list()
    p_values = [None] * len(gene)
    cell_type = None
    time_point = None
    control_list = list()
    treated_list = list()
    treated_indices = list()
    count_replicates = 0
    count_conditions = 1 # adds control group here
    
    #Go through each condition for each expression row
    for col, replicate in obs.iterrows():
        cell_type = replicate['cell_type']
        condition = replicate['condition']
        time_point = replicate['time_point']
        
        # Replicate is control group ('control', 'wildtype', 'input')
        if any(condition.lower() in c for c in control_string):
            # Add replicate to list
            if len(control_list) < grouped.loc[cell_type, condition, time_point]['replicate']: 
                # Still adding replicates of current condition
                control_list.append(gene[count_replicates])
        
        # Replicate is non-control
        else:
            # Do we already get all the replicates for this condition?
            if len(treated_list) < grouped.loc[cell_type, replicate['condition'], time_point]['replicate']:
                # Still adding replicates of current condition
                treated_list.append(gene[count_replicates])
                treated_indices.append(count_replicates)
                
        if len(control_list) == grouped.loc[cell_type, condition, time_point]['replicate'] and \
            len(treated_list) == grouped.loc[cell_type, replicate['condition'], time_point]['replicate']:        
            
            # Calculate pvalue
            import scipy.stats as stats
            p_coef, p_value = stats.pearsonr(control_list, treated_list)
            p_value = round(p_value, 7)
            #print('PEARSON COEFFICIENT: ', p_coef)
            
            #Controls are NaN, treated replicates ge the pvalue
#             for c in range(len(control_list)):
#                 p_values.append(None)
            for t in range(len(treated_list)):
                #p_values.append(p_value)
                p_values[treated_indices[t]] = p_value
                
            # Reset
            treated_list = list()
            treated_indices = list()
            if count_conditions == grouped.index.levels[1].size:
                control_list = list()
            
        count_replicates += 1

    p_values_list.append(p_values)
    print('p_values: ', p_values)

Xpval = np.array(p_values_list)

adata.Xpval = Xpval.T

#Return this:
#Xpval.T
# Xpval
            


p_values:  [None, None, None, 0.6836206, 0.6836206, 0.6836206, None, None, None, 0.5876048, 0.5876048, 0.5876048, None, None, None, 0.4909728, 0.4909728, 0.4909728]
p_values:  [None, None, None, 0.9686701, 0.9686701, 0.9686701, None, None, None, 0.55428, 0.55428, 0.55428, None, None, None, 0.0692434, 0.0692434, 0.0692434]
p_values:  [None, None, None, 0.9736786, 0.9736786, 0.9736786, None, None, None, 0.2822105, 0.2822105, 0.2822105, None, None, None, 0.6407919, 0.6407919, 0.6407919]
p_values:  [None, None, None, 0.4734865, 0.4734865, 0.4734865, None, None, None, 0.6760904, 0.6760904, 0.6760904, None, None, None, 0.6263417, 0.6263417, 0.6263417]
p_values:  [None, None, None, 0.5705433, 0.5705433, 0.5705433, None, None, None, 0.6414396, 0.6414396, 0.6414396, None, None, None, 0.6674344, 0.6674344, 0.6674344]
p_values:  [None, None, None, 0.986875, 0.986875, 0.986875, None, None, None, 0.4330716, 0.4330716, 0.4330716, None, None, None, 0.8511378, 0.8511378, 0.8511378]
p_values:  [None, N

p_values:  [None, None, None, 0.7117083, 0.7117083, 0.7117083, None, None, None, 0.8750934, 0.8750934, 0.8750934, None, None, None, 0.2793275, 0.2793275, 0.2793275]
p_values:  [None, None, None, 0.5486596, 0.5486596, 0.5486596, None, None, None, 0.6599638, 0.6599638, 0.6599638, None, None, None, 0.9887782, 0.9887782, 0.9887782]
p_values:  [None, None, None, 0.4747417, 0.4747417, 0.4747417, None, None, None, 0.1078032, 0.1078032, 0.1078032, None, None, None, 0.9978876, 0.9978876, 0.9978876]
p_values:  [None, None, None, 0.5019931, 0.5019931, 0.5019931, None, None, None, 0.5401869, 0.5401869, 0.5401869, None, None, None, 0.999155, 0.999155, 0.999155]
p_values:  [None, None, None, 0.4267174, 0.4267174, 0.4267174, None, None, None, 0.8674775, 0.8674775, 0.8674775, None, None, None, 0.1428268, 0.1428268, 0.1428268]
p_values:  [None, None, None, 0.1699257, 0.1699257, 0.1699257, None, None, None, 0.329274, 0.329274, 0.329274, None, None, None, 0.6729514, 0.6729514, 0.6729514]
p_values:  [None

In [29]:
###################################### Calculate FDR ##########################################################
# Yang: http://www.statsmodels.org/devel/generated/statsmodels.sandbox.stats.multicomp.multipletests.html

In [92]:
Xpval = adata.Xpval.T
# print(Xpval)

# from numpy import array, empty                                                                        
# pvalues = array(pvalues) 
# n = float(pvalues.shape[0])                                                                           
# new_pvalues = empty(n)

replicates = get_replicate_counts(adata.obs)

fdrs_list = list()
for index, row in enumerate(Xpval):
       
    current_fdrs = [None] * len(row)    
    start = 0
    end = 0
    pvalue_list = list()
    pvalue_indices = list() # tuple of int values (index_of_first_replicate, number_of_replicates)
    for i, rep_count in enumerate(replicates):
        end += rep_count

        # Take 1st replicate of group (skip None groups - controls)
        if row[start:end][0] is not None:
            pvalue_list.append(row[start:end][0])
            pvalue_indices.append((start, rep_count))
            
            
        start += rep_count
        
    n = len(pvalue_list)    
    fdr_list = list()

    pvalue_list = [ (pvalue, i) for i, pvalue in enumerate(pvalue_list) ]
    pvalue_list.sort()
    pvalue_list.reverse()
    for i, vals in enumerate(pvalue_list):
        rank = n - i
        pvalue, index = vals
        fdr_list.append((n/rank) * pvalue)
    for i in range(0, int(n)-1):
        if fdr_list[i] < fdr_list[i+1]:
            fdr_list[i+1] = fdr_list[i]
    for i, vals in enumerate(pvalue_list):
        pvalue, index = vals
        fdr_list[index] = fdr_list[i]
    
    
    # Add the FDR to FDR_list for each replicate. Control replicates remain None
    for (i, rep_count), fdr in zip(pvalue_indices, fdr_list):
        for t in range(rep_count):
            current_fdrs[i+t] = fdr
    
#     print('current_fdrs: ', current_fdrs)
    fdrs_list.append(current_fdrs)
        
# print(fdrs_list)
Xfdr = np.array(fdrs_list)

print(Xfdr)

#Return the transposed np.array
#return Xfdr.T

[[None None None ... 0.6836206 0.6836206 0.6836206]
 [None None None ... 0.20773019999999998 0.20773019999999998
  0.20773019999999998]
 [None None None ... 0.96118785 0.96118785 0.96118785]
 ...
 [None None None ... 0.1481253 0.1481253 0.1481253]
 [None None None ... 0.2926852 0.2926852 0.2926852]
 [None None None ... 0.9668941 0.9668941 0.9668941]]
